In [1]:
import json
from collections import defaultdict
import numpy as np

In [2]:
import plotly
plotly.tools.set_credentials_file(username='amcire96', api_key='sej35ud4YbSOfIshhhZg')
# plotly.tools.set_credentials_file(username='anni_rong', api_key='2pD1U5HLtMWzawdPrW4Z')
# print(plotly.__version__)

import plotly.plotly as py
from plotly.graph_objs import *
# import plotly.tools.figure_factory as FF
from plotly.tools import FigureFactory as FF
from plotly.tools import make_subplots
import requests
requests.packages.urllib3.disable_warnings()



In [3]:
with open("business_id_to_nounadj_vecs.json", "r") as f:
    bid_to_vecs = json.load(f)

In [4]:
bid_to_categories = {}
# c = 0
valid_business_ids =  bid_to_vecs.keys()
with open("biz_food.json", "r") as f:
    for line in f:
        business = json.loads(line)
        if business["business_id"] in valid_business_ids:
            bid_to_categories[business["business_id"]] = business["categories"]
#             c += 1


In [5]:
print(len(bid_to_categories))
print(len(bid_to_vecs))

9857
9857


In [6]:
for bid, (noun_vec, adj_vec) in bid_to_vecs.items()[:5]:
    print(noun_vec)

[[0.0025272200320748, 0.0, 0.0036892339664737154, 0.007083602858094484, 0.0, 0.00015242412758348516, 0.001937965978257641, 0.0033005142996056854, 0.0042452836926871055, 0.0025102202722274035, 0.0003293199417331103, 0.0, 0.007579379652648081, 0.0009426805038318943, 0.0028137983798559737, 0.0017851984208585197, 0.0, 0.02170163325027534, 0.0003083446290243459, 0.009093699035305722, 0.0, 0.0, 0.003215842202132176, 0.0, 0.0, 0.0009281528568270846, 0.0034225710917744454, 0.011024644523356195, 0.0, 0.0002072408519119697]]
[[0.00239464180530652, 0.00043249438207357304, 0.007976778639045322, 0.005887193485326345, 0.055554072510799114, 0.0005097079303660752, 0.0, 0.0005565951754705961, 0.00040733593484165595, 0.006318717823243197, 0.004055339458183562, 1.1646302556083485e-05, 0.002175675302443967, 0.004426308124160851, 0.003239435901318526, 0.0012427877534596083, 0.0013116746163421252, 0.0013695634237070136, 0.0030280238375782614, 0.002328229779646522, 0.0, 0.005917642465569493, 0.00304169844826

In [7]:
for bid, categories in bid_to_categories.items()[:5]:
    print(categories)

[u'Vietnamese', u'Restaurants']
[u'Burgers', u'American (New)', u'Restaurants']
[u'Fast Food', u'Poutineries', u'Restaurants']
[u'Bakeries', u'Event Planning & Services', u'Food', u'Desserts', u'Caterers']
[u'Tacos', u'Bars', u'Nightlife', u'Salad', u'Restaurants', u'Mexican', u'Cocktail Bars']


In [8]:
category_to_counts = {}
category_to_inds = defaultdict(list)
k=1
num_topics = 30
for bid in bid_to_categories.keys():
    categories = bid_to_categories[bid]
    ([noun_vec], [adj_vec]) = bid_to_vecs[bid]
#     print(noun_vec)
    top_inds = np.argsort(noun_vec)[::-1][:k]
#     print(top_inds)
    for category in categories:
        if category not in category_to_counts:
            category_to_counts[category] = np.zeros(num_topics)
        for ind in top_inds:
            category_to_counts[category][ind] += 1
            category_to_inds[category].append(ind)
#     print(categories)
print(category_to_counts)

    

{u'Drugstores': array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  3.,  6.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  4.,  0.,  0.,  0.,  0.,  0.,  0.,  6.,
        7.,  0.,  0.,  0.]), u'ATV Rentals/Tours': array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
        0.,  0.,  0.,  0.]), u'Buffets': array([  0.,   1.,   8.,   0.,   0.,   5.,   6.,   1.,   0.,   1.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   3.,   0.,   0.,   0.,   0.,
         1.,  55.,   0.,   0.,   0.,   0.,   0.,   0.]), u'Creperies': array([  0.,   0.,   1.,   7.,   1.,   0.,   0.,   9.,   0.,   0.,   1.,
         6.,   1.,  16.,   2.,   0.,   0.,   0.,   2.,   0.,   0.,   3.,
         1.,   0.,   1.,   1.,   0.,   1.,   0.,   0.]), u'Canadian (New)': array([ 12.,   3.,  36.,  17.,  17.,  28.,   1.,   3.,   0.,   0.,   4.,
        12.,   2.,   3.,   0.,   4.,  17.,  10.,   3.,  24.,   1.,  14.,
        51.,   1., 

In [9]:
cleaned_category_to_counts = {k:v for (k,v) in category_to_counts.items() if sum(v) >= 100}
cleaned_category_to_inds = {k:v for (k,v) in category_to_inds.items() if len(v) >= 100}
cleaned_category_to_counts

{u'American (New)': array([  17.,   17.,   90.,   18.,   50.,  126.,    3.,    7.,    1.,
           5.,    7.,   31.,    0.,    7.,    1.,   13.,   42.,   10.,
           3.,   24.,   16.,   30.,  114.,   13.,   97.,   10.,    3.,
           3.,    9.,    4.]),
 u'American (Traditional)': array([   6.,    9.,  108.,   20.,   77.,  132.,    3.,    2.,    0.,
           4.,    9.,   56.,    1.,    4.,    4.,    8.,   41.,    9.,
           3.,   15.,    8.,   42.,   20.,   13.,   64.,   12.,    3.,
           3.,   40.,    2.]),
 u'Arts & Entertainment': array([  10.,    1.,   25.,   38.,    6.,  144.,    2.,   10.,    1.,
          12.,    1.,    0.,    0.,    4.,    0.,    0.,   34.,    2.,
           1.,    1.,    3.,    5.,    2.,    9.,    5.,   37.,    3.,
           0.,    1.,    2.]),
 u'Asian Fusion': array([  1.,   1.,  44.,  15.,   5.,   7.,  51.,   2.,   0.,   0.,  10.,
          2.,   1.,   2.,   1.,   1.,   3.,  59.,   1.,   6.,   3.,   3.,
         26.,   5.,   2.,   3., 

In [10]:
print(len(cleaned_category_to_counts))
print(len(cleaned_category_to_inds))

63
63


In [11]:
topic_keywords = [
    "table server night experience people",
    "pizza crust pizzas toppings slice",
    "food atmosphere portions quality family",
    "place people friends family places",
    "burger fries burgers bacon bun",
    "bar drinks drink bartender area",
    "sushi rolls roll fish sashimi",
    "coffee shop starbucks cup latte",
    "staff atmosphere experience customers owner",
    "time times wait visit today",
    "chicken rice waffles curry orange",
    "breakfast eggs pancakes toast bacon",
    "service customer times stars notch",
    "cream ice chocolate tea flavors",
    "order minutes delivery drink phone",
    "lunch dinner specials today spot",
    "beer selection beers wine tap",
    "sauce rice beef pork soup",
    "location times store locations area",
    "restaurant family restaurants dishes area",
    "tacos salsa chips taco burrito",
    "sandwich sandwiches bread cheese fries",
    "menu items options things dishes",
    "buffet crab buffets selection line",
    "salad steak bread meal cheese",
    "vegas las strip hotel places",
    "prices portions store hour quality",
    "price quality portions portion taste",
    "wings sauce fries buffalo bbq",
    "thai pad curry tea spicy"]


In [12]:
# num_plots = len(cleaned_category_to_inds)
start_ind = 0
num_plots=6

fig = make_subplots(rows=num_plots, cols=1)


for i, (category, counts) in enumerate(cleaned_category_to_counts.items()[start_ind:start_ind+num_plots]):
    trace = Bar(x=range(len(counts)), y=counts, name=category, text=topic_keywords)
    fig.append_trace(trace, i+1, 1)

# fig['layout'].update(height=800)
    
py.iplot(fig)



This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]
[ (5,1) x5,y5 ]
[ (6,1) x6,y6 ]



In [13]:
start_ind = 6
num_plots= 6

fig = make_subplots(rows=num_plots, cols=1)


for i, (category, counts) in enumerate(cleaned_category_to_counts.items()[start_ind:start_ind+num_plots]):
    trace = Bar(x=range(len(counts)), y=counts, name=category, text=topic_keywords)
    fig.append_trace(trace, i+1, 1)

# fig['layout'].update(height=800)
    
py.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]
[ (5,1) x5,y5 ]
[ (6,1) x6,y6 ]



In [14]:
start_ind = 12
num_plots= 6

fig = make_subplots(rows=num_plots, cols=1)


for i, (category, counts) in enumerate(cleaned_category_to_counts.items()[start_ind:start_ind+num_plots]):
    trace = Bar(x=range(len(counts)), y=counts, name=category, text=topic_keywords)
    fig.append_trace(trace, i+1, 1)

# fig['layout'].update(height=800)
    
py.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]
[ (5,1) x5,y5 ]
[ (6,1) x6,y6 ]



In [15]:
start_ind = 18
num_plots= 6

fig = make_subplots(rows=num_plots, cols=1)


for i, (category, counts) in enumerate(cleaned_category_to_counts.items()[start_ind:start_ind+num_plots]):
    trace = Bar(x=range(len(counts)), y=counts, name=category, text=topic_keywords)
    fig.append_trace(trace, i+1, 1)

# fig['layout'].update(height=800)
    
py.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]
[ (5,1) x5,y5 ]
[ (6,1) x6,y6 ]



In [16]:
start_ind = 24
num_plots= 6

fig = make_subplots(rows=num_plots, cols=1)


for i, (category, counts) in enumerate(cleaned_category_to_counts.items()[start_ind:start_ind+num_plots]):
    trace = Bar(x=range(len(counts)), y=counts, name=category, text=topic_keywords)
    fig.append_trace(trace, i+1, 1)

# fig['layout'].update(height=800)
    
py.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]
[ (5,1) x5,y5 ]
[ (6,1) x6,y6 ]



In [17]:
start_ind = 30
num_plots= 6

fig = make_subplots(rows=num_plots, cols=1)


for i, (category, counts) in enumerate(cleaned_category_to_counts.items()[start_ind:start_ind+num_plots]):
    trace = Bar(x=range(len(counts)), y=counts, name=category, text=topic_keywords)
    fig.append_trace(trace, i+1, 1)

# fig['layout'].update(height=800)
    
py.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]
[ (5,1) x5,y5 ]
[ (6,1) x6,y6 ]



In [18]:
start_ind = 36
num_plots= 6

fig = make_subplots(rows=num_plots, cols=1)


for i, (category, counts) in enumerate(cleaned_category_to_counts.items()[start_ind:start_ind+num_plots]):
    trace = Bar(x=range(len(counts)), y=counts, name=category, text=topic_keywords)
    fig.append_trace(trace, i+1, 1)

# fig['layout'].update(height=800)
    
py.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]
[ (5,1) x5,y5 ]
[ (6,1) x6,y6 ]



In [19]:
start_ind = 42
num_plots= 6

fig = make_subplots(rows=num_plots, cols=1)


for i, (category, counts) in enumerate(cleaned_category_to_counts.items()[start_ind:start_ind+num_plots]):
    trace = Bar(x=range(len(counts)), y=counts, name=category, text=topic_keywords)
    fig.append_trace(trace, i+1, 1)

# fig['layout'].update(height=800)
    
py.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]
[ (5,1) x5,y5 ]
[ (6,1) x6,y6 ]



In [20]:
start_ind = 48
num_plots= 6

fig = make_subplots(rows=num_plots, cols=1)


for i, (category, counts) in enumerate(cleaned_category_to_counts.items()[start_ind:start_ind+num_plots]):
    trace = Bar(x=range(len(counts)), y=counts, name=category, text=topic_keywords)
    fig.append_trace(trace, i+1, 1)

# fig['layout'].update(height=800)
    
py.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]
[ (5,1) x5,y5 ]
[ (6,1) x6,y6 ]



In [21]:
start_ind = 54
num_plots= 6

fig = make_subplots(rows=num_plots, cols=1)


for i, (category, counts) in enumerate(cleaned_category_to_counts.items()[start_ind:start_ind+num_plots]):
    trace = Bar(x=range(len(counts)), y=counts, name=category, text=topic_keywords)
    fig.append_trace(trace, i+1, 1)

# fig['layout'].update(height=800)
    
py.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]
[ (5,1) x5,y5 ]
[ (6,1) x6,y6 ]



In [22]:
start_ind = 60
num_plots= 3

fig = make_subplots(rows=num_plots, cols=1)


for i, (category, counts) in enumerate(cleaned_category_to_counts.items()[start_ind:start_ind+num_plots]):
    trace = Bar(x=range(len(counts)), y=counts, name=category, text=topic_keywords)
    fig.append_trace(trace, i+1, 1)

# fig['layout'].update(height=800)
    
py.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]

